In [51]:
# !pip install git+https://github.com/ImalshaD/DimenTuna.git

In [52]:
import torch

In [53]:
torch.set_float32_matmul_precision('high')

In [54]:
from dimentuna import DTHfEncoder, LayerWrappebleDTHfLLM, LayerWrappebleQwen, DTConfig

In [55]:
from dimentuna import LinearMapper, DTLayerWrapper, LinearWrapper, FeedForwardProjector, LSTMProjector

In [56]:
from dimentuna import TwoPhasedTS, TwoPhasedSeq2SeqTS

In [57]:
from dimentuna import DTMsgm

In [58]:
from datasets import load_dataset

In [59]:
from torch.utils.data import Dataset, DataLoader

# Parameters

In [60]:
batch_size = 4
epochs = 10

In [61]:
data_cache = '\data_cache'

In [62]:
model_cache = ".\model_cache"

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datasets and Preprocessing

In [64]:
mgsm = DTMsgm()

In [65]:
ds = load_dataset("nthakur/GSM8KInstruct-Parallel-instruct")

In [66]:
train_ds = ds['train']
test_ds = ds['test']

In [67]:
train_df = train_ds.to_pandas()[["prompt", "code"]]
test_df = test_ds.to_pandas()[["prompt", "code"]]

In [68]:
train_df[["System_prompt", "Instruction"]] = train_df['prompt'].str.split('### Instruction:', expand=True)
test_df[["System_prompt", "Instruction"]] = test_df['prompt'].str.split('### Instruction:', expand=True)

In [69]:
train_df = train_df[["Instruction", "code"]]
test_df = test_df[["Instruction", "code"]]

In [70]:
train_df["Instruction"] = train_df["Instruction"].str.strip()
test_df["Instruction"] = test_df["Instruction"].str.strip()

In [71]:
train_df['Instruction'] = train_df['Instruction'].str.rsplit('?', n=1).str[0] + '?'
test_df["Instruction"] = test_df["Instruction"].str.rsplit('?', n=1).str[0] + '?'

In [72]:
# Step 1: Add a column with the length of each "Instruction"
train_df["Instruction_Length"] = train_df["Instruction"].apply(len)
test_df["Instruction_Length"] = test_df["Instruction"].apply(len)

# Step 2: Determine a threshold (e.g., 95th percentile of length)
train_threshold = train_df["Instruction_Length"].quantile(0.97)
test_threshold = test_df["Instruction_Length"].quantile(0.97)

# Step 3: Filter out unusually large rows
train_df = train_df[train_df["Instruction_Length"] <= train_threshold]
test_df = test_df[test_df["Instruction_Length"] <= test_threshold]

# Step 4: Drop the temporary "Instruction_Length" column
train_df = train_df.drop(columns=["Instruction_Length"])
test_df = test_df.drop(columns=["Instruction_Length"])

In [74]:
train_df_english = train_df[train_df['code']=='en'].reset_index(drop=True)
test_df_english = test_df[test_df['code']=='en'].reset_index(drop=True)

In [75]:
train_df_thai = train_df[train_df['code']=='th'].reset_index(drop=True)
test_df_thai = test_df[test_df['code']=='th'].reset_index(drop=True)

In [76]:
train_df_english

,Instruction,code
0,John has 2 umbrellas in his house and 1 in the...,en
1,Brad wanted to set up a lemonade stand to earn...,en
2,"Angus, Patrick, and Ollie went fishing for tro...",en
3,Frank has an apple tree in his backyard. 5 app...,en
4,"Ann, Bill, Cate, and Dale each buy personal pa...",en
...,...,...
6895,If Jade earns $1600 per month and spent 75% of...,en
6896,John builds a toy bridge to support various we...,en
6897,"Coral reads 30 pages of a book on night 1, and...",en
6898,Janet has 24 dresses. Half of them have pocke...,en


In [77]:
test_df_english

,Instruction,code
0,Julian has 400 legos and wants to make lego mo...,en
1,"Mark has 30 candies, Peter has 25 candies, and...",en
2,"Marj has two $20 bills, three $5 bills, and $4...",en
3,Hunter saw 5 frogs sitting on top lily pads in...,en
4,A portable computer drive has enough kilobytes...,en
...,...,...
360,"Cappuccinos cost $2, iced teas cost $3, cafe l...",en
361,Jamal works at a library shelving books. He ha...,en
362,"Out of the 200 students in a class, thirty per...",en
363,Albert has to run 99 meters in gym class. The ...,en


In [78]:
test_df_english['Instruction'][0]

'Julian has 400 legos and wants to make lego models of two identical airplanes. If each airplane model requires 240 legos, how many more legos does Julian need?'

In [79]:
train_df_thai

,Instruction,code
0,ควินซีเพิ่งซื้อรถยนต์โดยใช้สินเชื่อ 5 ปีโดยไม่...,th
1,พีเตอร์กำลังเก็บใบไม้ร่วง ใช้เวลา 15 นาทีในการ...,th
2,บิลได้รับทีวีจอแบนที่มีขนาด 48 นิ้ว x 100 นิ้ว...,th
3,พอลได้รับเงิน 12.50 ดอลลาร์ต่อชั่วโมงที่เขาทำง...,th
4,เจมส์ซื้อเนื้อสเต็กแบบซื้อ 1 แถม 1 ราคาคือ 15 ...,th
...,...,...
7017,Jeanette กำลังฝึกการโยนลูกบอล ในแต่ละสัปดาห์เธ...,th
7018,ลอร่าต้องซื้อวัสดุประดับหน้าต่างสำหรับหน้าต่าง...,th
7019,โทมัสกำลังหวังที่จะวิ่งมาราธอนในปีหน้า ซึ่งมีร...,th
7020,เพจซื้อสติกเกอร์ใหม่และต้องการแบ่งปันกับเพื่อน...,th


In [80]:
test_df_thai['Instruction'][0]

'โครงการคอนโดใหม่นี้มีชั้นทั้งหมด 23 ชั้น ชั้นปกติมีหน่วยที่อยู่ 12 หน่วย ในขณะที่ชั้นพานเพนท์เฮาส์มีหน่วยเพียง 2 หน่วย เมื่อมีการจัดสรรชั้นบนสุด 2 ชั้นสำหรับหน่วยพานท์เฮาส์ จำนวนหน่วยทั้งหมดในคอนโดนี้เป็นเท่าใด?'

**DataLoaders**

In [81]:
train_english_loader = DataLoader(train_df_english['Instruction'], batch_size=batch_size)
test_english_loader = DataLoader(test_df_english["Instruction"], batch_size=batch_size)

In [82]:
train_thai_loader = DataLoader(train_df_thai['Instruction'], batch_size=batch_size//2)
test_thai_loader = DataLoader(test_df_thai['Instruction'], batch_size=batch_size//2)

In [83]:
xlmr_config = DTConfig(
    model_name="FacebookAI/xlm-roberta-base",
    cache_dir="./cache",          # Path to cache directory
    max_tokens=512,               # Maximum token limit
    temperature=0.7,              # Sampling temperature
    max_generation_length=256,    # Maximum length for generated text
    do_sample=True,               # Enable sampling
    truncation=True,              # Enable truncation
    padding=True,                 # Enable padding
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),  # Device configuration
    use_best_config=False,        # Use best configuration (custom logic)
    padding_side='right',         # Padding side
    output_hidden_states=False,   # Output hidden states
    output_attentions=False,      # Output attention values
    return_dict=True,             # Return output as dictionary
)

In [84]:
xlmr = DTHfEncoder(xlmr_config)

In [85]:
test = ["This is test Sentence.", "This is test sentence 2.", "This is the longest test sentence 3."]

In [86]:
input_ten = xlmr.encode(test)

# Models

In [87]:
mapper = LinearMapper(1536, 1536, 1024)

In [88]:
qwen_model = LayerWrappebleQwen()

In [89]:
qwen_model.set_system_prompt(mgsm.system_prompt)

In [90]:
qwen_model.print_best_config()

{'max_length': 512}


In [91]:
qwen_ed = qwen_model.get_embeddings_size()

In [92]:
xlmr_ed = xlmr.get_embeddings_size()

In [93]:
projector = LSTMProjector(qwen_ed, xlmr_ed, 1024)

In [94]:
qwen_model.get_Layer_output(test, 1).shape

torch.Size([3, 41, 1536])

In [95]:
layer_1_wrapper= LinearWrapper(mapper)

In [96]:
qwen_model.replace_layer(1, layer_1_wrapper)

In [97]:
qwen_model.print_status()

-LLM
--Layer_0
---Layer_Frozen: False
--LayerWrapper_1
---Layer_Frozen: False
---Mapper_Frozen: False
---Engaged: True
---Alpha_Frozen: False Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: False
--Layer_3
---Layer_Frozen: False
--Layer_4
---Layer_Frozen: False
--Layer_5
---Layer_Frozen: False
--Layer_6
---Layer_Frozen: False
--Layer_7
---Layer_Frozen: False
--Layer_8
---Layer_Frozen: False
--Layer_9
---Layer_Frozen: False
--Layer_10
---Layer_Frozen: False
--Layer_11
---Layer_Frozen: False
--Layer_12
---Layer_Frozen: False
--Layer_13
---Layer_Frozen: False
--Layer_14
---Layer_Frozen: False
--Layer_15
---Layer_Frozen: False
--Layer_16
---Layer_Frozen: False
--Layer_17
---Layer_Frozen: False
--Layer_18
---Layer_Frozen: False
--Layer_19
---Layer_Frozen: False
--Layer_20
---Layer_Frozen: False
--Layer_21
---Layer_Frozen: False
--Layer_22
---Layer_Frozen: False
--Layer_23
---Layer_Frozen: False
--Layer_24
---Layer_Frozen: False
--Layer_25
---Layer_Frozen: False
--Layer_26
---Layer_Frozen: Fa

In [98]:
two_phased_ts = TwoPhasedSeq2SeqTS(
    llm=qwen_model,
    encoder=xlmr,
    projector=projector,
    train_loader=train_english_loader,
    val_loader=test_english_loader,
    lr=0.001,
    device=device,
    target_layers=[1],
    mapper_train_loader=train_thai_loader,
    mapper_val_loader=test_thai_loader,
    layer_shift=3
)

In [99]:
qwen_model.freeze()

In [100]:
qwen_model.print_status()

-LLM
--Layer_0
---Layer_Frozen: True
--LayerWrapper_1
---Layer_Frozen: True
---Mapper_Frozen: True
---Engaged: True
---Alpha_Frozen: True Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: True
--Layer_3
---Layer_Frozen: True
--Layer_4
---Layer_Frozen: True
--Layer_5
---Layer_Frozen: True
--Layer_6
---Layer_Frozen: True
--Layer_7
---Layer_Frozen: True
--Layer_8
---Layer_Frozen: True
--Layer_9
---Layer_Frozen: True
--Layer_10
---Layer_Frozen: True
--Layer_11
---Layer_Frozen: True
--Layer_12
---Layer_Frozen: True
--Layer_13
---Layer_Frozen: True
--Layer_14
---Layer_Frozen: True
--Layer_15
---Layer_Frozen: True
--Layer_16
---Layer_Frozen: True
--Layer_17
---Layer_Frozen: True
--Layer_18
---Layer_Frozen: True
--Layer_19
---Layer_Frozen: True
--Layer_20
---Layer_Frozen: True
--Layer_21
---Layer_Frozen: True
--Layer_22
---Layer_Frozen: True
--Layer_23
---Layer_Frozen: True
--Layer_24
---Layer_Frozen: True
--Layer_25
---Layer_Frozen: True
--Layer_26
---Layer_Frozen: True
--Layer_27
---Layer_Froze

In [101]:
two_phased_ts.train(epochs)

Encoder_Frozen: True
Projector_Frozen: False
-LLM
--Layer_0
---Layer_Frozen: True
--LayerWrapper_1
---Layer_Frozen: True
---Mapper_Frozen: True
---Engaged: False
---Alpha_Frozen: True Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: True
--Layer_3
---Layer_Frozen: True
--Layer_4
---Layer_Frozen: True
--Layer_5
---Layer_Frozen: True
--Layer_6
---Layer_Frozen: True
--Layer_7
---Layer_Frozen: True
--Layer_8
---Layer_Frozen: True
--Layer_9
---Layer_Frozen: True
--Layer_10
---Layer_Frozen: True
--Layer_11
---Layer_Frozen: True
--Layer_12
---Layer_Frozen: True
--Layer_13
---Layer_Frozen: True
--Layer_14
---Layer_Frozen: True
--Layer_15
---Layer_Frozen: True
--Layer_16
---Layer_Frozen: True
--Layer_17
---Layer_Frozen: True
--Layer_18
---Layer_Frozen: True
--Layer_19
---Layer_Frozen: True
--Layer_20
---Layer_Frozen: True
--Layer_21
---Layer_Frozen: True
--Layer_22
---Layer_Frozen: True
--Layer_23
---Layer_Frozen: True
--Layer_24
---Layer_Frozen: True
--Layer_25
---Layer_Frozen: True
--Layer_26
-

100%|██████████| 92/92 [00:03<00:00, 30.08it/s]


Epoch 0 : Loss 0.002681309810394178 Eval Loss 0.001620857551208009


100%|██████████| 92/92 [00:03<00:00, 30.07it/s]


Epoch 1 : Loss 0.0016492022191970676 Eval Loss 0.0015394819733362806


...

Epoch 7 : Loss 0.0020408072571273615 Eval Loss 0.0020182937100479328


100%|██████████| 185/185 [00:05<00:00, 31.69it/s]


Epoch 8 : Loss 0.0020406021785568757 Eval Loss 0.0020182937100479328


100%|██████████| 185/185 [00:05<00:00, 31.59it/s]


Epoch 9 : Loss 0.0020406021785568757 Eval Loss 0.0020182937100479328


In [102]:
qwen_model.print_status()

-LLM
--Layer_0
---Layer_Frozen: True
--LayerWrapper_1
---Layer_Frozen: True
---Mapper_Frozen: False
---Engaged: True
---Alpha_Frozen: False Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: True
--Layer_3
---Layer_Frozen: True
--Layer_4
---Layer_Frozen: True
--Layer_5
---Layer_Frozen: True
--Layer_6
---Layer_Frozen: True
--Layer_7
---Layer_Frozen: True
--Layer_8
---Layer_Frozen: True
--Layer_9
---Layer_Frozen: True
--Layer_10
---Layer_Frozen: True
--Layer_11
---Layer_Frozen: True
--Layer_12
---Layer_Frozen: True
--Layer_13
---Layer_Frozen: True
--Layer_14
---Layer_Frozen: True
--Layer_15
---Layer_Frozen: True
--Layer_16
---Layer_Frozen: True
--Layer_17
---Layer_Frozen: True
--Layer_18
---Layer_Frozen: True
--Layer_19
---Layer_Frozen: True
--Layer_20
---Layer_Frozen: True
--Layer_21
---Layer_Frozen: True
--Layer_22
---Layer_Frozen: True
--Layer_23
---Layer_Frozen: True
--Layer_24
---Layer_Frozen: True
--Layer_25
---Layer_Frozen: True
--Layer_26
---Layer_Frozen: True
--Layer_27
---Layer_Fro

In [103]:
qwen_model.engage_all_layer_wrappers(False)

In [104]:
mgsm.evaluate(qwen_model,['en', 'th'], batch_size=batch_size, logging=True)

Evaluating en:   2%|▏         | 1/63 [00:03<04:01,  3.90s/it]

answer: 1.5, correct: 3 correctness: False
answer: 280, correct: 320 correctness: False
answer: 500, correct: 500 correctness: True
answer: 3.43, correct: 6 correctness: False


Evaluating en:   3%|▎         | 2/63 [00:06<02:55,  2.88s/it]

answer: 120, correct: 90 correctness: False
answer: Empty Answer, correct: 57 correctness: False
answer: 28, correct: 28 correctness: True
answer: 140, correct: 140 correctness: True


Evaluating en:   5%|▍         | 3/63 [00:09<03:06,  3.11s/it]

...

Evaluating th:  98%|█████████▊| 62/63 [03:44<00:03,  3.57s/it]

answer: Empty Answer, correct: 36 correctness: False
answer: Empty Answer, correct: 48 correctness: False
answer: 3, correct: 70 correctness: False
answer: Empty Answer, correct: 5 correctness: False


Evaluating th: 100%|██████████| 63/63 [03:47<00:00,  3.62s/it]

answer: 30, correct: 30 correctness: True
answer: 165, correct: 175 correctness: False


{'en': {'accuracy': 0.6190476190476191},
 'th': {'accuracy': 0.21031746031746032}}

In [105]:
qwen_model.engage_layer_wrapper(1)

In [106]:
qwen_model.print_status()

-LLM
--Layer_0
---Layer_Frozen: True
--LayerWrapper_1
---Layer_Frozen: True
---Mapper_Frozen: False
---Engaged: True
---Alpha_Frozen: False Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: True
--Layer_3
---Layer_Frozen: True
--Layer_4
---Layer_Frozen: True
--Layer_5
---Layer_Frozen: True
--Layer_6
---Layer_Frozen: True
--Layer_7
---Layer_Frozen: True
--Layer_8
---Layer_Frozen: True
--Layer_9
---Layer_Frozen: True
--Layer_10
---Layer_Frozen: True
--Layer_11
---Layer_Frozen: True
--Layer_12
---Layer_Frozen: True
--Layer_13
---Layer_Frozen: True
--Layer_14
---Layer_Frozen: True
--Layer_15
---Layer_Frozen: True
--Layer_16
---Layer_Frozen: True
--Layer_17
---Layer_Frozen: True
--Layer_18
---Layer_Frozen: True
--Layer_19
---Layer_Frozen: True
--Layer_20
---Layer_Frozen: True
--Layer_21
---Layer_Frozen: True
--Layer_22
---Layer_Frozen: True
--Layer_23
---Layer_Frozen: True
--Layer_24
---Layer_Frozen: True
--Layer_25
---Layer_Frozen: True
--Layer_26
---Layer_Frozen: True
--Layer_27
---Layer_Fro

In [107]:
mgsm.evaluate(qwen_model,['en', 'th'], batch_size=batch_size, logging=True)

Evaluating en:   2%|▏         | 1/63 [00:05<05:32,  5.37s/it]

answer: Empty Answer, correct: 36 correctness: False
answer: Empty Answer, correct: 150 correctness: False
answer: Empty Answer, correct: 15 correctness: False
answer: Empty Answer, correct: 80 correctness: False


Evaluating en:   3%|▎         | 2/63 [00:10<05:22,  5.28s/it]

answer: Empty Answer, correct: 10 correctness: False
answer: Empty Answer, correct: 320 correctness: False
answer: Empty Answer, correct: 5 correctness: False
answer: Empty Answer, correct: 60 correctness: False


Evaluating en:   5%|▍         | 3/63 [00:16<05:23,  5.40s/it]

...

Evaluating th:  98%|█████████▊| 62/63 [04:16<00:04,  4.57s/it]

answer: Empty Answer, correct: 10 correctness: False
answer: Empty Answer, correct: 694 correctness: False
answer: Empty Answer, correct: 30 correctness: False
answer: Empty Answer, correct: 90 correctness: False


Evaluating th: 100%|██████████| 63/63 [04:20<00:00,  4.14s/it]

answer: Empty Answer, correct: 14 correctness: False
answer: Empty Answer, correct: 22 correctness: False


{'en': {'accuracy': 0.0}, 'th': {'accuracy': 0.0}}

In [110]:
mgsm.print_sample(qwen_model,"th", 32)

Query: แมรีเป็นผู้ที่รักการทำสวนเป็นชีวิตจิตใจ เมื่อวานนี้เธอได้รับต้นไม้ใหม่ 18 กระถางจากโรงเพาะที่เธอชื่นชอบ ตามขอบหน้าต่าง 40 บานที่บ้านในชนบทของเธอมีต้นไม้ตั้งอยู่แล้วบานละ 2 กระถาง ด้วยความรู้สึกอยากแบ่งปัน เธอได้ตัดสินใจจะมอบต้นไม้ 1 กระถางบนขอบหน้าต่างแต่ละบานให้เพื่อนๆ และครอบครัวในวันพรุ่งนี้ แมรีจะเหลือต้นไม่กี่กระถาง 
 Correct Answer: 58 
 Generated Answer: .sup indeed   (app.concat chick当!--atureitionalshaw.main chick chief__( chief只见shawʲouvomba promote chiefature   licit妻itional Chick chief indeed_Resource   ʲ af    ectmp chicking   言λ!--   ouv chickHomepage当 chickouvouv(appouv床上.mainʲouvouv second   loy   (prom妻 chick chick effet refute两个月 chick官网(appimate    Promʲ显.main chickʲ chickouv(app在床上ouvouvاتفouvouv promotionouv_Resource妻(apping.main豺ouvombalicit indeed chickouv妻!--   licit otrauitenshawouvouvouvʲouv妻妻 chick chick(appouvshaw af chick chickHomepage然ouvouv泰山itionalouv邑.concat(app(app chick(app.concatouvouv(app床上.mainouvosten.main chickʲouvʲshawʲouv IllegalArgument

In [111]:
qwen_model.engage_all_layer_wrappers(False)

In [112]:
mgsm.print_sample(qwen_model,"th", 32)

Query: ผู้ดูแลสัตว์ให้อาหารเอปทั้งหมดในสวนสัตว์ โดยทุก 2 เดือนเขาจะสั่งซื้อกล้วยทั้งหมดจากสวนในท้องถิ่น หากในแต่ละเดือน ลิงต้องการกล้วย 200 ลูก กอริลลาต้องการกล้วย 400 ลูก และบาบูนต้องการกล้วย 100 ลูก เขาต้องสั่งซื้อกล้วยกี่ลูกจึงจะเพียงพอสำหรับ 2 เดือน 
 Correct Answer: 1400 
 Generated Answer: 1. จำนวนลูกที่จำเป็นในการสั่งซื้อกล้วยในแต่ละเดือน:
   - ลิง: 200 ลูก/เดือน
   - โกริลลา: 400 ลูก/เดือน
   - บาบูน: 100 ลูก/เดือน

2. รวมจำนวนลูกที่จำเป็นในการสั่งซื้อกล้วยใน 2 เดือน:
   200 + 400 + 100 = 700 ลูก

3. คำตอบ: ผู้ดูแลสัตว์ต้องสั่งซื้อกล้วย 700 ลูกเพียงพอสำหรับ 2 เดือน.

Final Answer: [700] 
 Extracted Answer: 700 

Query: คุณยายโจนส์อบพายแอปเปิ้ล 5 ถาดสำหรับงานเลี้ยงมื้อเที่ยงเจ้าหน้าที่ดับเพลิง เธอตัดพายแต่ละถาดออกเป็น 8 ชิ้นและวางพายห้าถาดนี้ไว้บนโต๊ะบุฟเฟต์เพื่อให้แขกตักเอง เมื่อจบงาน หลังจากที่แขกได้ตักพายของตัวเองไปกินแล้ว พายยังเหลืออยู่อีก 14 ชิ้น แขกตักพายไปกี่ชิ้น 
 Correct Answer: 26 
 Generated Answer: ขั้นตอนการวิเคราะห์:

1. คุณยายโจนส์ทำพายจำนวน 5 ถาด
2. ถูกต้องว่าแต